# **Subset geospatial data and calculate landscape metrics**

By Bridget Bittmann

Date created: March 14, 2022

Date editted: March 28, 2022

In [1]:
## IMPORT PACKAGES ##

import numpy as np #basic computation
!pip install geopandas
import geopandas as gpd #geopandas for .shp
import matplotlib.pyplot as plt #to create plots
import pandas as pd #to create dataframes and export .csv
!pip install rasterio
import rasterio as rso #import GeoTiff files
from rasterio.mask import mask #to crop data to a boundary
from rasterio.plot import show #to plot the image
from rasterio.crs import CRS
from shapely.ops import unary_union #creates boundary of shapefile
import json #imports metadata
!pip install rioxarray #to clip rasters to a .shp file
import rioxarray as rxr
from rasterio.warp import calculate_default_transform, reproject, Resampling
!pip install pylandstats
import pylandstats #to perform landscape metrics
from pylandstats import landscape
from pylandstats import SpatioTemporalAnalysis #to calculate landscape metrics through time
import glob
import os
import matplotlib.lines as lines
import matplotlib.patches as patch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 6.3 MB 20.5 MB/s 
     |████████████████████████████████| 16.7 MB 253 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 42.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47 kB 2.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for rioxarray: filename=rioxarray-0.9.1-py3-none-any.whl size=54611 sha256=61207713d93fca7448cdba83e1f1f29cdba7daede7cba8cec70c02ca6473bcf0
  Stored in directory: /root/.cache/pip/wheels/07/da/9e/1cc57b2e7a29a206893db83e984a341e2e94378263e0798229
Successfully built rioxarr

Module /usr/local/lib/python3.7/dist-packages/pylandstats/landscape.py has not been compiled for Transonic-Pythran


In [2]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
## NAVIGATE TO YOUR DIRECTORY ##
%cd gdrive/MyDrive/spatial_colab/datasets/
%ls

/content/gdrive/MyDrive/spatial_colab/datasets
climate_stats/         irrigation_companies/  lcmap_files/   subset_test_shp/
diversion_timeseries/  irrig_lbrb/            masked/
extra_figures/         IrrMapper/             output_files/
hydromet_data/         LBRB_shp/              POUs/


In [ ]:
## ---------------------- ##
## SUBSET GEOSPATIAL DATA ##
## ---------------------- ##

shp_file = gpd.read_file('POUs/POUs_EDIT_060622_Merge.shp') #open shapefile
names = shp_file['WaterRight']
files = glob.glob('lcmap_files/*.tiff') #get all the years of cdl imagery
data =[]
for i in range(len(files)):
  data.append(rso.open(files[i])) #open cdl image and append to a list
shp = shp_file.to_crs(data[1].crs) #reproject the shp file to same projection
years = np.arange(1987, 2021) #years of LCMAP data
collection = []
for i in range(len(shp)):
  for n in range(len(years)):
    dataset = data[n]
    year_out = dataset.name[21:-15]
    extent = gpd.GeoSeries(shp['geometry'][i]) #get the geometry from shapefile
    coords = [json.loads(extent.to_json())['features'][0]['geometry']] #gets coordinates for rasterio input
    out_img, out_transform = mask(dataset=data[n], shapes=coords, crop=True, nodata=0) #crop the data to the shapefile
    out_meta = data[n].meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": out_img.shape[1],
                     "width": out_img.shape[2],
                     "transform": out_transform})
    # Merge original file name with init_landcover to denote that it is the initial land cover data for Janus
    in_file = files[n]
    out_filename = os.path.join('lcmap_files/lcmap_masked/'+names[i]+'_'+year_out+'.tif') #create a file name to export to
    # Save clipped land cover coverage THIS WILL OVERWRITE FILES
    out_tiff = rso.open(out_filename, 'w', **out_meta)
    out_tiff.write(np.squeeze(out_img, 0), 1)
    out_tiff.close()
    collection.append(out_img)

In [5]:
## ------------------------------ ##
## SUBSET GEOSPATIAL DATA TO BPBC ##
## ------------------------------ ##

shp_file = gpd.read_file('subset_test_shp/BPBC_Divisions/BPBC_Divisions-IDTM.shp') #open shapefile
names = shp_file['DivisionNa']
files = glob.glob('lcmap_files/*.tiff') #get all the years of cdl imagery
data =[]
for i in range(len(files)):
  data.append(rso.open(files[i])) #open cdl image and append to a list
shp = shp_file.to_crs(data[1].crs) #reproject the shp file to same projection
years = np.arange(1987, 2021) #years of LCMAP data
collection = []
for i in range(len(shp)):
  for n in range(len(years)):
    dataset = data[n]
    year_out = dataset.name[21:-15]
    extent = gpd.GeoSeries(shp['geometry'][i]) #get the geometry from shapefile
    coords = [json.loads(extent.to_json())['features'][0]['geometry']] #gets coordinates for rasterio input
    out_img, out_transform = mask(dataset=data[n], shapes=coords, crop=True, nodata=0) #crop the data to the shapefile
    out_meta = data[n].meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": out_img.shape[1],
                     "width": out_img.shape[2],
                     "transform": out_transform})
    # Merge original file name with init_landcover to denote that it is the initial land cover data for Janus
    in_file = files[n]
    out_filename = os.path.join('lcmap_files/lcmap_masked_bpbc/'+names[i]+'_'+year_out+'.tif') #create a file name to export to
    # Save clipped land cover coverage THIS WILL OVERWRITE FILES
    out_tiff = rso.open(out_filename, 'w', **out_meta)
    out_tiff.write(np.squeeze(out_img, 0), 1)
    out_tiff.close()
    collection.append(out_img)

In [ ]:
## ---------------------------------------- ##
## Import multiple rasters into PyLandStats ##
## ---------------------------------------- ##
years = np.arange(1987,2021)
temporal_group = []
for i in range(len(names)):
  files= sorted(glob.glob('lcmap_files/lcmap_masked/'+names[i]+'_*.tif')) #name for all the csv files
  sta = SpatioTemporalAnalysis(files, dates=years, nodata=0) #import all CDL rasters and mask
  temporal_group.append(sta)

In [6]:
## ---------------------------------------- ##
## Import multiple rasters into PyLandStats ##
## ---------------------------------------- ##
years = np.arange(1987,2021)
temporal_group = []
for i in range(len(names)):
  files= sorted(glob.glob('lcmap_files/lcmap_masked_bpbc/'+names[i]+'_*.tif')) #name for all the csv files
  sta = SpatioTemporalAnalysis(files, dates=years, nodata=0) #import all CDL rasters and mask
  temporal_group.append(sta)

In [7]:
# ------------------------------- #
# CALCULATE THE CLASS PROPORTIONS #
# ------------------------------- #

proportions = []

for i in range(len(names)):
  df = SpatioTemporalAnalysis.compute_class_metrics_df(temporal_group[i], metrics=['proportion_of_landscape'])
  df.to_csv('lcmap_files/proportions/'+names[i]+'_prop.csv')
  proportions.append(df)

In [8]:
# ------------------------------------ #
# CALCULATE THE CLASS PROPORTIONS BPBC #
# ------------------------------------ #

proportions = []

for i in range(len(names)):
  df = SpatioTemporalAnalysis.compute_class_metrics_df(temporal_group[i], metrics=['proportion_of_landscape'])
  df.to_csv('lcmap_files/proportions/bpbc_metrics/long/'+names[i]+'_prop.csv')
  proportions.append(df)

In [ ]:
# ------------------------------- #
# CALCULATE CONFIGURATION METRICS #
# ------------------------------- #

config = []

for i in range(len(names)):
  df = SpatioTemporalAnalysis.compute_landscape_metrics_df(temporal_group[i], metrics = ['contagion', 'largest_patch_index'])
  df.to_csv('lcmap_files/configuration/'+names[i]+'_configuration.csv')
  config.append(df)

/usr/local/lib/python3.7/dist-packages/pylandstats/landscape.py:2604: RuntimeWarning: Entropy-based metrics can only be computed in landscapes with more than two classes of patches. Returning nan
  RuntimeWarning,


In [9]:
# ------------------------------------ #
# CALCULATE CONFIGURATION METRICS BPBC #
# ------------------------------------ #

config = []

for i in range(len(names)):
  df = SpatioTemporalAnalysis.compute_landscape_metrics_df(temporal_group[i], metrics = ['contagion', 'largest_patch_index'])
  df.to_csv('lcmap_files/proportions/bpbc_metrics/long/'+names[i]+'_configuration.csv')
  config.append(df)

In [ ]:
# ----------------------------------------------------------------- #
# Put class proportions in the same format as configuration metrics #
# ----------------------------------------------------------------- #

# Import csv files into a list of dataframes

files = sorted(glob.glob('lcmap_files/proportions/*_prop.csv'))
files_config = sorted(glob.glob('lcmap_files/configuration/*_configuration.csv'))
names = list(sorted(shp_file['WaterRight']))

proportions = []
for i in files:
  data = pd.read_csv(i)
  proportions.append(data)
config = []
for i in files_config:
  data = pd.read_csv(i)
  config.append(data)

#Create new dataframes in same format as configuration metrics

new_df = []
for i in range(len(proportions)):
  df = pd.DataFrame(years, columns=['dates'])
  prop = proportions[i]
  df['DivName'] = names[i]
  df['class1_urban'] = prop['proportion_of_landscape'][prop['class_val'] == 1]
  df['class2_crops'] = prop['proportion_of_landscape'][prop['class_val'] == 2].values
  df = df.fillna(0)
  new_df.append(df)


In [10]:
# ---------------------------------------------------------------------- #
# Put class proportions in the same format as configuration metrics BPBC #
# ---------------------------------------------------------------------- #

# Import csv files into a list of dataframes

files = sorted(glob.glob('lcmap_files/proportions/bpbc_metrics/long/*_prop.csv'))
files_config = sorted(glob.glob('lcmap_files/proportions/bpbc_metrics/long/*_configuration.csv'))
names = list(sorted(shp_file['DivisionNa']))

proportions = []
for i in files:
  data = pd.read_csv(i)
  proportions.append(data)
config = []
for i in files_config:
  data = pd.read_csv(i)
  config.append(data)

#Create new dataframes in same format as configuration metrics

new_df = []
for i in range(len(proportions)):
  df = pd.DataFrame(years, columns=['dates'])
  prop = proportions[i]
  df['DivName'] = names[i]
  df['class1_urban'] = prop['proportion_of_landscape'][prop['class_val'] == 1]
  df['class2_crops'] = prop['proportion_of_landscape'][prop['class_val'] == 2].values
  df = df.fillna(0)
  new_df.append(df)


In [ ]:
## ------------------------------------------ ##
## CALCULATE CHANGE IN URBAN AREA FOR MAPPING ##
## ------------------------------------------ ##

prop = pd.concat(new_df)

change = prop.groupby('DivName', as_index=False).class1_urban.agg(['min','max']).reset_index().fillna(0)
change['urb_change'] = change['max']-change['min']
change.to_csv('lcmap_files/proportions/change.csv')

In [12]:
## ----------------------------------------------- ##
## CALCULATE CHANGE IN URBAN AREA FOR MAPPING BPBC ##
## ----------------------------------------------- ##

prop = pd.concat(new_df)

change = prop.groupby('DivName', as_index=False).class1_urban.agg(['min','max']).reset_index().fillna(0)
change['urb_change'] = change['max']-change['min']
change.to_csv('lcmap_files/proportions/bpbc_change.csv')

In [15]:
## -------------------- ##
## MERGE TWO DATAFRAMES ##
## -------------------- ##

merged = []

for i in range(len(new_df)):
  df = new_df[i]
  con = config[i]
  df_merge = df.merge(con, on='dates', how='left')
  df_merge.to_csv('lcmap_files/final_metrics/bpbc/'+names[i]+'.csv')
  merged.append(df_merge)